### Overview
This notebook gives a first analysis of the EIS data. First, the data is processed and all spectra are plotted. Then for the NX001 battery, the data is interpolated to create a full data frame with synthetic EIS data for every cycle. Finally, infrastructure for testing is built, although there is insufficient data to perform interpolation while keeping some data for testing.

Functions to remove zeros from the data and plot the Nyquist diagrams:

In [1]:
import pandas as pd
import os
import plotly.graph_objects as go

def remove_zeros(file_path, create_new_file):
    df = pd.read_csv(file_path, sep='\s+', header=None, skiprows=1)

    df = df.apply(pd.to_numeric, errors='coerce')

    #Set column names
    df.columns = ["|Z|/Ohm", "freq/Hz", "Re(Z)/Ohm", "-Im(Z)/Ohm", "cycle number"]

    #Remove zeros
    df_filtered = df[(df[['|Z|/Ohm', 'freq/Hz', 'Re(Z)/Ohm', '-Im(Z)/Ohm', 'cycle number']] != 0).all(axis=1)]

    if create_new_file == True:
        base_name = os.path.basename(file_path)
        name, ext = os.path.splitext(base_name)
        output_file_name = f"{name}_clean{ext}"
        output_file_path = os.path.join(os.path.dirname(file_path), output_file_name)
        df_filtered.to_csv(output_file_path, sep='\t', index=False, header=False)

    column_headers = df.iloc[0]

    return df_filtered

def plot_spectrum(df, battery_name, show_frequency_gradient):
    cycle_column, re_column, im_column, freq_column = 'cycle number', 'Re(Z)/Ohm', '-Im(Z)/Ohm', 'freq/Hz'
    fig = go.Figure()
    
    cycles = df[cycle_column].unique()
    traces = []
    buttons = []

    for i, cycle in enumerate(cycles):
        df_cycle = df[df[cycle_column] == cycle]
        if show_frequency_gradient:
            marker = dict(
                size=5.5, 
                color=df_cycle[freq_column], 
                colorscale='Viridis', 
                colorbar=dict(
                    title='Frequency (Hz)',
                    thickness=15,
                    len=0.5
                )
            )
        else:
            marker = dict(
                size=5.5,
            )
        
        trace = go.Scatter(
            x=df_cycle[re_column], 
            y=df_cycle[im_column], 
            mode='markers', 
            name=f'Cycle {int(cycle)}', 
            marker=marker
        )
        traces.append(trace)

    initial_visibility = [False] * len(cycles)
    for i, cycle in enumerate(cycles):
        button = dict(
            method='update',
            label=f'Cycle {int(cycle)}',
            visible=True,
            args=[{'visible': [(not visibility) if j == i else visibility for j, visibility in enumerate(initial_visibility)]},
                  {'title': f'Impedance Plot'}]
        )
        buttons.append(button)

    all_button = dict(method='update', label='All', visible=True, args=[{'visible': [True] * len(traces)}])
    buttons.append(all_button)

    fig.update_layout(
        autosize=False, 
        width=700, 
        height=600,
        updatemenus=[
            dict(
                type='buttons',
                direction='right',
                x=1.1,
                y=1.1,
                buttons=buttons
            )
        ],
        title=f'Impedance Plot of Battery {battery_name}',
        xaxis_title='Re(Z)/Ohm',
        yaxis_title='-Im(Z)/Ohm',
        showlegend=True,
        legend=dict(orientation='v', x=1.02, y=1)
    )

    for trace in traces:
        fig.add_trace(trace)

    fig.show()


Import all the EIS data and remove zeros:

In [2]:
file_paths = [
    ('Data/EIS_Data/NX001_2108_EIS.txt'),
    ('Data/EIS_Data/NX001_2401_EIS.txt'),
    ('Data/EIS_Data/NX002_2108_EIS.txt'),
    ('Data/EIS_Data/NX006_2108_EIS.txt'),
    ('Data/EIS_Data/NX006_2401_EIS.txt'),
    ('Data/EIS_Data/RS001_2108_EIS.txt'),
    ('Data/EIS_Data/RS006_2108_EIS.txt'),
    ('Data/EIS_Data/RS006_2401_EIS.txt'),
    ('Data/EIS_Data/SG003_2401_EIS.txt'),
    ('Data/EIS_Data/SG004_2401_EIS.txt')
]

clean_data = []

for file_path in file_paths:
    df = remove_zeros(file_path, False)
    battery_name = os.path.basename(file_path).split('_EIS.txt')[0]
    clean_data.append((df, battery_name)) 

df_NX001_2108, battery_name_NX001_2108 = clean_data[0]
df_NX001_2401, battery_name_NX001_2401 = clean_data[1]
df_NX002_2108, battery_name_NX002_2108 = clean_data[2]
df_NX006_2108, battery_name_NX006_2108 = clean_data[3]
df_NX006_2401, battery_name_NX006_2401 = clean_data[4]
df_RS001_2108, battery_name_RS001_2108 = clean_data[5]
df_RS006_2108, battery_name_RS006_2108 = clean_data[6]
df_RS006_2401, battery_name_RS006_2401 = clean_data[7]
df_SG003_2401, battery_name_SG003_2401 = clean_data[8]
df_SG004_2401, battery_name_SG004_2401 = clean_data[9]

### Plot the Nyquist diagrams
**Note** you can change from showing a frequency gradient on the Nyquist diagrams to just having block colours for easier comparison by changing the last input to: `show_frequency_gradient=False`.

In [11]:
for df, battery_name in clean_data:
    plot_spectrum(df, battery_name, show_frequency_gradient=False)

remove_zeros('Data/EIS_Data/NX001_2108_EIS.txt', True)

,|Z|/Ohm,freq/Hz,Re(Z)/Ohm,-Im(Z)/Ohm,cycle number
5489,0.043669,10001.000000,0.032931,-0.028680,1.0
5490,0.039612,7913.000500,0.033556,-0.021050,1.0
5491,0.037781,6262.001000,0.034568,-0.015247,1.0
5492,0.037329,4954.999000,0.035744,-0.010762,1.0
5493,0.037627,3919.999500,0.036871,-0.007502,1.0
...,...,...,...,...,...
240909,0.068713,0.025611,0.067688,0.011822,154.0
240910,0.069905,0.020955,0.068760,0.012602,154.0
240911,0.072621,0.016298,0.071194,0.014324,154.0
240912,0.075260,0.011642,0.073619,0.015630,154.0


### Notes on plots
Firstly, it should be noted that the EIS data is potentiostatic, which means that the impedance data was found by applying a constant potential to the working electrode for a certain amount of time.

If the cell has the date(?) 2108, it was EIS test was performed at 4 points: in cycle 1, 52, 103 and 154. If it has the date(?) 2401 then EIS was performed once at cycle 1, except for the two SG batteries which were tested on cycle 20. 

Important: The 2401 marked tests are all performed on batteries that have previously been tested in the 2108 test, does this mean that cycle 20 is actually cycle X+20, where X is the number of cycles the battery is put through in the 2108 test? Under this assumption, I have found the total number of cycles each battery goes through, see below.

**Why are there two lines for each EIS test?**
This is especially apparent for the SG batteries, where these lines are very far apart. This is because the EIS test is actually performed twice for each cycle: frequencies from 10kHz to 0Hz are applied and then there is a break and this is repeated. Why was this done, is it useful? Why are the two tests for the same cycle so far apart for the SG batteries?

Also important: Because Nicholas showed us that a specific pattern of current and voltage was performed (i.e. sections with constant voltage), will this affect the EIS and our analysis? Will analyse cycling data next to get a better idea.

### First impressions
The plots are extremely dissimilar between different battery types. Hence, it is clear that different ECMs will be needed to model the different battery types.

However, on all plots the expected key features of the impedance spectrum are visible: the low frequency semi-circle and the high frequency asymptote can be identified.

### Thoughts on how to proceed

One issue is that the data we have is limited. We have the most data for the NX battery. One route we could take is a data driven model, where we use NX001, NX002 and NX006 data to generate synthetic data. However, because the data is from the same cycle numbers this isn't ideal (it would be better if we had a wider range of cycle numbers). Ideally we would train on all the data and then run EIS on another battery at cycle numbers other than 1, 52, 103 and 154 to test if our model can accurately return the estimated cycle number. This would be possible, as Rob did say we could do some testing on their batteries.

If we can create a model that can return cycle number, we next need to correlate this to battery degradation, which is where an ECM may step in, or we could use the charging data to get a relationship for cycle number and SOH. If we did have an ECM for the NX battery, we could use non-linear least squares fitting to find the circuit parameters that fit best with the data we have. The success of NLLS would be dependent on how accurate the ECM is to the physical cell, and it would also be limited by the data we have. We would need a different ECM for each battery type - we probably don't have time for this. We should ask LiFETIME which battery is most important, or stick with NX since we have the most data for that battery type.

In [4]:
file_paths = [
    ('Data/EIS_Data/NX001_2108_EIS.txt'),
    ('Data/EIS_Data/NX001_2401_EIS.txt'),
    ('Data/EIS_Data/NX002_2108_EIS.txt'),
    ('Data/EIS_Data/NX006_2108_EIS.txt'),
    ('Data/EIS_Data/NX006_2401_EIS.txt'),
    ('Data/EIS_Data/RS001_2108_EIS.txt'),
    ('Data/EIS_Data/RS006_2108_EIS.txt'),
    ('Data/EIS_Data/RS006_2401_EIS.txt'),
    ('Data/EIS_Data/SG003_2401_EIS.txt'),
    ('Data/EIS_Data/SG004_2401_EIS.txt')
]

for path in file_paths:
    df = pd.read_csv(path, sep='\s+', header=None, skiprows=1)
    df.columns = ["|Z|/Ohm", "freq/Hz", "Re(Z)/Ohm", "-Im(Z)/Ohm", "cycle number"]
    total_cycles = df.iloc[-1, -1]
    battery_name = os.path.basename(path).split('_EIS.txt')[0] 
    print(f'Battery {battery_name} went through {int(total_cycles)} cycles.')

Battery NX001_2108 went through 169 cycles.
Battery NX001_2401 went through 49 cycles.
Battery NX002_2108 went through 171 cycles.
Battery NX006_2108 went through 124 cycles.
Battery NX006_2401 went through 40 cycles.
Battery RS001_2108 went through 175 cycles.
Battery RS006_2108 went through 175 cycles.
Battery RS006_2401 went through 52 cycles.
Battery SG003_2401 went through 32 cycles.
Battery SG004_2401 went through 38 cycles.


Total number of cycles for each battery, and on which cycle EIS was performed:

| Syntax      | Description | EIS 1  | EIS 2  | EIS 3  | EIS 4  | EIS 5  | 
| ----------- | ----------- | ------ | ------ | ------ | ------ | ------ |
| NX001       | 218         | 1      | 52     | 103    | 154    | 170    |
| NX002       | 171         | 1      | 52     | 103    | 154    | -      |
| NX006       | 164         | 1      | 52     | 103    | **125**| -      |
| RS001       | 175         | 1      | 52     | 103    | 154    | -      |
| RS006       | 227         | 1      | 52     | 103    | 154    | 176    |
| SG003       | 32          | 20     | -      | -      | -      | -      |
| SG004       | 38          | 20     | -      | -      | -      | -      |

It should be noted that there is a cycle 0... this should be discussed to check a zero error hasn't been made here.

### Generating synthetic EIS data for NX001 from the available EIS data

In this next section, I generated artificial EIS data to sit between the cycles we have EIS data for (cycles 1, 52, 103 and 154). I used the SciPy interpolation package to perform simple linear interpolation between the cycles. This is not ideal, but because of the limited data its a best guess at what lies in the gaps. A selection of resulting spectra can be seen plotted below. Note, I only use the first EIS test data for the cycle.

In [5]:
import numpy as np
from scipy.interpolate import interp1d

#Remove duplicates by averaging
df_NX001_2108 = df_NX001_2108.drop_duplicates(subset=['cycle number', 'freq/Hz']).reset_index(drop=True)

#Extract unique cycles and frequencies
cycles = [1, 52, 103, 154]
frequencies = df_NX001_2108['freq/Hz'].unique()

synthetic_data = []

#Interpolation function
def interpolate_cycle_data(df, freq, cycle_numbers, target_cycle):
    re_z_values = []
    im_z_values = []
    
    for f in freq[::-1]:
        df_freq = df[df['freq/Hz'] == f]
        if df_freq.empty:
            continue
        
        re_z = df_freq.pivot(index='cycle number', columns='freq/Hz', values='Re(Z)/Ohm')
        im_z = df_freq.pivot(index='cycle number', columns='freq/Hz', values='-Im(Z)/Ohm')
        
        interp_re = interp1d(cycle_numbers, re_z.values.flatten(), kind='linear', fill_value='extrapolate')
        interp_im = interp1d(cycle_numbers, im_z.values.flatten(), kind='linear', fill_value='extrapolate')
        re_z_values.append(interp_re(target_cycle))
        im_z_values.append(interp_im(target_cycle))
    
    return re_z_values, im_z_values

In [6]:
#Generate 'synthetic' data for cycles 1 to 200
for cycle in range(1, 201):
    re_z, im_z = interpolate_cycle_data(df_NX001_2108, frequencies, cycles, cycle)
    for f, re, im in zip(frequencies[::-1], re_z, im_z):
        if not np.isnan(re) and not np.isnan(im):
            magnitude = np.sqrt(re**2 + im**2)
            synthetic_data.append([magnitude, f, re, im, cycle])

NX_synthetic_data = pd.DataFrame(synthetic_data, columns=['|Z|/Ohm', 'freq/Hz', 'Re(Z)/Ohm', '-Im(Z)/Ohm', 'cycle number'])

In [7]:
def plot_select_cycles(df, cycles_to_plot):
    cycle_column, re_column, im_column = 'cycle number', 'Re(Z)/Ohm', '-Im(Z)/Ohm'
    fig = go.Figure()
    
    traces = []
    buttons = []

    for cycle in cycles_to_plot:
        df_cycle = df[df[cycle_column] == cycle]
        trace = go.Scatter(x=df_cycle[re_column], y=df_cycle[im_column], mode='markers', name=f'Cycle {cycle}', marker=dict(size=5.5))
        traces.append(trace)

    initial_visibility = [False] * len(cycles_to_plot)
    for i, cycle in enumerate(cycles_to_plot):
        button = dict(method='update',
                      label=f'Cycle {cycle}',
                      visible=True,
                      args=[{'visible': [(not visibility) if j == i else visibility for j, visibility in enumerate(initial_visibility)]},
                            {'title': f'Impedance Plot for Cycle {cycle}'}])
        buttons.append(button)

    all_button = dict(method='update',label='All',visible=True,args=[{'visible': [True] * len(traces)}, {'title': 'Impedance Plot for All Cycles'}])
    buttons.append(all_button)

    fig.update_layout(autosize=False, width=800, height=700,
        updatemenus=[
            dict(
                type='buttons',
                direction='right',
                x=1.1,
                y=1.08,
                buttons=buttons
            )
        ],
        title='Impedance plot showing selected cycles of generated data from NX001',
        xaxis_title='Re(Z)/Ohm',
        yaxis_title='-Im(Z)/Ohm',
        showlegend=True,
        legend=dict(orientation='v', x=1.02, y=1)
    )

    for trace in traces:
        fig.add_trace(trace)

    fig.show()


In [8]:
cycles_to_plot = [1, 25, 52, 75, 103, 125, 154, 175]
plot_select_cycles(NX_synthetic_data, cycles_to_plot)

The goal of this exercise was to produce a complete EIS dataset for future use. The next step would be to figure out how EIS links to SOH, and my thoughts to do this from a data-driven angle would be to look at the correlating data for NX001 and look at how SOH changes with cycle number.

The following code was written with the aim of using the data from the NX001_2401 and NX006_2401 files to test the accuracy of the interpolation. However, this was under the assumption that cycle 1 in these files actually corresponds with cycle 169+1 and so forth. Annoyingly, it is clearly visible from the earlier plots that this would not work, as you can see that cycle 154 from 2108 is not comparable to the 2401 data, (note the scale of the plot). It could well still be true that cycle 1 corresponds with 169+1, with the difference being due to the fact that the battery degraded in the period between tests, either just from not being used or maybe it was used in this time but the data wasn't recorded. You can see from the decreasing slope of the MSE plots that it seems like the data from the NX001_2401 and NX006_2401 files corresponds to a battery that has undergone more than 200 cycles. I've left it here as a good reminder that cycle number is NOT an indicator of SOH. Unfortunately, we do not have any data to test the interpolation. 

ACTION: get info from LiFETIME on what the 2401 data is.

In [9]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import plotly.graph_objects as go

# Remove duplicates
df_NX001_2401 = df_NX001_2401.drop_duplicates(subset=['cycle number', 'freq/Hz']).reset_index(drop=True)
df_NX006_2401 = df_NX006_2401.drop_duplicates(subset=['cycle number', 'freq/Hz']).reset_index(drop=True)

#Function to compute MSE
def compute_mse(df_real, df_synthetic, cycle_num):
    df_real=df_real[df_real['cycle number'] == 1.0]
    df_synthetic=df_synthetic[df_synthetic['cycle number'] == cycle_num].reset_index(drop=True)
    abs_mse = ((df_real['|Z|/Ohm'] - df_synthetic['|Z|/Ohm']) ** 2).mean()
    return abs_mse

#Plot comparison of real and synthetic data
def plot_comparison(df_real, df_synthetic, cycle_num):
    df_real=df_real[df_real['cycle number'] == 1.0]
    df_synthetic=df_synthetic[df_synthetic['cycle number'] == cycle_num]
    
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df_real['Re(Z)/Ohm'], y=df_real['-Im(Z)/Ohm'], mode='markers', name='Real Data', marker=dict(size=5.5)))
    fig.add_trace(go.Scatter(x=df_synthetic['Re(Z)/Ohm'], y=df_synthetic['-Im(Z)/Ohm'], mode='markers', name='Synthetic Data', marker=dict(size=5.5)))

    fig.update_layout(title=f'Comparison of Real and Synthetic Data for Cycle {cycle_num}',
                      xaxis_title='Re(Z)/Ohm',
                      yaxis_title='-Im(Z)/Ohm',
                      showlegend=True)

    fig.show()

cycle_num = 170
plot_comparison(df_NX001_2401, NX_synthetic_data, cycle_num)
abs_mse_170 = compute_mse(df_NX001_2401, NX_synthetic_data, cycle_num)
print(f"MSE between df_NX001_2401 cycle 1 and synthetic data cycle {cycle_num} = {abs_mse_170}")

cycle_num = 125
plot_comparison(df_NX006_2401, NX_synthetic_data, cycle_num)
abs_mse_125 = compute_mse(df_NX006_2401,NX_synthetic_data, cycle_num)
print(f"MSE between df_NX006_2401 cycle 1 and synthetic data cycle {cycle_num} = {abs_mse_170}")


MSE between df_NX001_2401 cycle 1 and synthetic data cycle 170 = 0.003944472723599616


MSE between df_NX006_2401 cycle 1 and synthetic data cycle 125 = 0.003944472723599616


In [10]:
mse_list=[]
for cycle_num in range(1,200):
    abs_mse = compute_mse(df_NX001_2401, NX_synthetic_data, cycle_num)
    mse_list.append(abs_mse)

fig = go.Figure(data=go.Scatter(x=np.arange(1, 200), y=mse_list, mode='lines+markers'))
fig.update_layout(title='Mean Squared Error (MSE) vs. Cycle Number for NX001_2401 compared to the synthetic data',
                  xaxis_title='Cycle Number',
                  yaxis_title='MSE',
                  showlegend=False)
fig.show()

mse_list=[]
for cycle_num in range(1,200):
    abs_mse = compute_mse(df_NX006_2401, NX_synthetic_data, cycle_num)
    mse_list.append(abs_mse)

fig = go.Figure(data=go.Scatter(x=np.arange(1, 200), y=mse_list, mode='lines+markers'))
fig.update_layout(title='Mean Squared Error (MSE) vs. Cycle Number for NX006_2401 compared to the synthetic data',
                  xaxis_title='Cycle Number',
                  yaxis_title='MSE',
                  showlegend=False)
fig.show()

### Next steps - data-driven approach

As I mentioned, the next step would be to figure out how EIS links to SOH. This could be done from a data-driven angle, by looking at the correlating data for NX001 and seeing how SOH changes with cycle number. The idea would be that the EIS spectrum of a battery could be recorded. Then using the complete dataset (which would need to be expanded and improved), an estimate of the cycle number of the battery could be found. This wouldn't necessarily reflect the number of cycles the battery had been used for, but rather the effective cycle number corresponding to its level of degradation. Then, using the established relationship between SOH and cycle number found from cycling data, the corresponding state of health could be estimated. This raises many questions, not least because it relies overwhelmingly on the quality of the data, which may not correspond well with real-life data. Indeed, because of the size of the dataset, this approach won't be a good option. However, we could explore this route in order to provide learnings and recommendations to LiFETIME, that they could apply on a much larger scale in the future. 

This said, the data-driven approach would be unable to give any information of the physical degradation modes present in the battery, which is not ideal for the applications of the model. Ideally, we want to know more than just the SOH. 
